In [7]:
# for  generating data for images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# using adam as optimizer
from tensorflow.keras.optimizers import Adam
#cnn2d and max pooling to decrese time taken and for optimization purposes
from keras.layers import Convolution2D, MaxPooling2D
#dropout to avoid overfitting
from keras.layers import Dense, Dropout
#normalizes the contributions to a layer for every mini-batch
from keras.layers import Flatten, BatchNormalization
# to exit the model if the accuracy isnt incresing
from tensorflow.keras.callbacks import EarlyStopping
# model we are going to use
from keras.models import Sequential
# to list images
import os
# to better work with arrays
import numpy as np
# to use tensors and other tensorflow variables
import tensorflow as tf
# to get imgae data for test
from PIL import Image
# to see progress 
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [8]:
# rescaling the pixels and splitting them into train test
train_datagen = ImageDataGenerator(rescale = 1.0/255, validation_split=0.2)

# making images to generators preapering them to train
train_generator = train_datagen.flow_from_directory(
                                                    "imgs/train",
                                                    target_size = (200, 100),
                                                    class_mode = 'categorical',
                                                    batch_size = 64,
                                                    subset='training'
                                                    )
                                                    
# making images to generators preapering them to train      
validation_generator = train_datagen.flow_from_directory(
                                                    "imgs/train",
                                                    target_size=(200, 100),
                                                    batch_size=64,
                                                    class_mode='categorical',
                                                    subset='validation')

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [10]:
model = Sequential()

## CNN 1
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(200, 100, 3)))
model.add(BatchNormalization())
model.add(Convolution2D(32,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))

## CNN 2
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))

## CNN 3
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.5))

## Output
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 198, 98, 32)       896       
_________________________________________________________________
batch_normalization_7 (Batch (None, 198, 98, 32)       128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 198, 98, 32)       9248      
_________________________________________________________________
batch_normalization_8 (Batch (None, 198, 98, 32)       128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 99, 49, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 99, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 99, 49, 64)       

In [11]:
# stop early
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)
# model traning
model.fit(train_generator,epochs = 5,batch_size =64,validation_data=validation_generator,callbacks = [es],verbose = 1)

Epoch 1/5
281/281 [==============================] - 1660s 6s/step - loss: 1.2108 - acc: 0.6133 - val_loss: 4.2658 - val_acc: 0.4628
Epoch 2/5
281/281 [==============================] - 1662s 6s/step - loss: 0.4660 - acc: 0.8520 - val_loss: 0.6312 - val_acc: 0.8395
Epoch 3/5
281/281 [==============================] - 1655s 6s/step - loss: 0.4102 - acc: 0.8792 - val_loss: 0.3458 - val_acc: 0.9324
Epoch 4/5
281/281 [==============================] - 1658s 6s/step - loss: 0.2730 - acc: 0.9208 - val_loss: 0.1628 - val_acc: 0.9721
Epoch 5/5
281/281 [==============================] - 1650s 6s/step - loss: 0.1839 - acc: 0.9484 - val_loss: 2.5986 - val_acc: 0.9462


In [16]:
# save the model
# model.save("best_model_1.h5")

# load the model
from keras.models import load_model
model = load_model('best_model_1.h5')

In [17]:
# used to generate text according to predicted number
class_dict = {0 : "safe driving",
              1 : "texting - right",
              2 : "talking on the phone - right",
              3 : "texting - left",
              4 : "talking on the phone - left",
              5 : "operating the radio",
              6 : "drinking",
              7 : "reaching behind",
              8 : "hair and makeup",
              9 : "talking to passenger"}

In [ ]:
# funtion to convert image to array using pil
def image_to_array(file_path):
    img = Image.open(file_path)
    img = img.resize((200, 100))
    data = np.asarray(img,dtype='float32')
    data= tf.reshape(data, (-1,200, 100, 3))
    return data

In [18]:
#store predictions
predicted={}

# loop over images and predict its class
for i in tqdm(range(10000)):
    image=os.listdir("imgs/test")[i]
    img_path=f"imgs/test/{image}"
    img=image_to_array(img_path)
    y_pred=model.predict(img)
    y_pred=class_dict[np.argmax(y_pred)]
    predicted[image]=y_pred

100%|██████████| 10000/10000 [20:00<00:00,  8.33it/s]
